# MCP-Enabled Verification

This notebook demonstrates how to configure and run verification with MCP
(Model Context Protocol) tools. MCP turns the answering model into an **agent**
that can use external tools — web search, database queries, file operations —
before producing its final response.

| Step | What You'll Do |
|------|---------------|
| 1 | Configure MCP tools on a model |
| 2 | Set up agent middleware (limits, retry, summarization) |
| 3 | Run MCP-enabled verification |
| 4 | Inspect agent traces and tool usage |
| 5 | Handle recursion limit hits |
| 6 | Control trace input for evaluation |

See [MCP-Enabled Verification](../06-running-verification/mcp-verification.md)
for full documentation.

In [ ]:
# Mock cell: patches run_verification so examples execute without live API keys
# or running MCP servers. This cell is hidden in the rendered documentation.
import datetime
import os
from unittest.mock import patch

from karenina.schemas.results import VerificationResultSet
from karenina.schemas.verification import VerificationConfig, VerificationResult
from karenina.schemas.verification.model_identity import ModelIdentity
from karenina.schemas.verification.result_components import (
    VerificationResultMetadata,
    VerificationResultTemplate,
)

os.chdir(os.path.dirname(os.path.abspath("__file__")))


def _mock_run_verification(self, config, question_ids=None, **_kwargs):
    """Return realistic mock results simulating MCP agent verification."""
    qids = question_ids or self.get_question_ids()
    mock_results = []

    # Simulate agent performance: most pass, one hits recursion limit
    for qid in qids:
        q = self.get_question(qid)
        q_text = q.get("question", "")
        has_template = self.has_template(qid)

        for ans_model in config.answering_models:
            for parse_model in config.parsing_models:
                # Simulate: prime number question hits recursion limit
                hit_recursion = "prime number" in q_text
                passed = not hit_recursion if has_template else None

                answering = ModelIdentity(
                    interface=ans_model.interface,
                    model_name=ans_model.model_name,
                )
                parsing = ModelIdentity(
                    interface=parse_model.interface,
                    model_name=parse_model.model_name,
                )

                # Build a mock agent trace with tool calls
                if ans_model.mcp_urls_dict:
                    tool_names = list(ans_model.mcp_urls_dict.keys())
                    trace_lines = [
                        f"[Agent] Received question: {q_text}",
                        f"[Agent] Using tools from: {', '.join(tool_names)}",
                        "[Tool Call] web_search('relevant query')",
                        "[Tool Result] Found 3 results...",
                        "[Agent] Based on the search results, the answer is...",
                    ]
                    if hit_recursion:
                        trace_lines.append("[Agent] RECURSION LIMIT REACHED — returning partial response")
                    raw_response = "\n".join(trace_lines)
                else:
                    raw_response = f"Mock answer for: {q_text}"

                template_result = None
                if has_template:
                    template_result = VerificationResultTemplate(
                        raw_llm_response=raw_response,
                        verify_result=passed,
                        template_verification_performed=True,
                        recursion_limit_reached=hit_recursion,
                    )

                metadata = VerificationResultMetadata(
                    question_id=qid,
                    template_id=(self.get_template(qid)[:10] + "..." if has_template else "no_template"),
                    completed_without_errors=True,
                    question_text=q_text,
                    answering=answering,
                    parsing=parsing,
                    execution_time=3.5 if ans_model.mcp_urls_dict else 1.2,
                    timestamp=datetime.datetime.now(datetime.UTC).isoformat(),
                    result_id=f"mock-mcp-{qid[:8]}",
                )

                mock_results.append(
                    VerificationResult(
                        metadata=metadata,
                        template=template_result,
                    )
                )

    return VerificationResultSet(results=mock_results)


_patcher1 = patch(
    "karenina.benchmark.benchmark.Benchmark.run_verification",
    _mock_run_verification,
)
_patcher2 = patch(
    "karenina.schemas.verification.config.VerificationConfig._validate_config",
    lambda _self: None,
)
_ = _patcher1.start()
_ = _patcher2.start()

## Step 1: Configure MCP Tools

MCP tools are configured on `ModelConfig` via `mcp_urls_dict`. Each key is a
server name, each value is the MCP endpoint URL. Setting this field activates
agent mode for that model.

Use `mcp_tool_filter` to restrict which tools are available, and
`mcp_tool_description_overrides` to improve tool descriptions.

In [2]:
from karenina.schemas import ModelConfig

# Configure an answering model with MCP tools
answering_model = ModelConfig(
    id="agent-claude",
    model_name="claude-sonnet-4-5-20250929",
    model_provider="anthropic",
    interface="langchain",
    mcp_urls_dict={
        "search": "http://localhost:3000/mcp",
        "database": "http://localhost:3001/mcp",
    },
    mcp_tool_filter=["web_search", "query_db"],
    mcp_tool_description_overrides={
        "web_search": "Search the web for current biomedical research.",
        "query_db": "Query the genomics database for gene information.",
    },
)

print(f"Model: {answering_model.model_name}")
print(f"MCP servers: {list(answering_model.mcp_urls_dict.keys())}")
print(f"Tool filter: {answering_model.mcp_tool_filter}")
print(f"Agent mode: {answering_model.mcp_urls_dict is not None}")

Model: claude-sonnet-4-5-20250929
MCP servers: ['search', 'database']
Tool filter: ['web_search', 'query_db']
Agent mode: True


## Step 2: Agent Middleware Configuration

When MCP is enabled, the model runs as an agent — making multiple LLM calls
and invoking tools between calls. `AgentMiddlewareConfig` controls safety
limits, retry behavior, summarization, and prompt caching.

In [3]:
from karenina.schemas.config.models import (
    AgentLimitConfig,
    AgentMiddlewareConfig,
    ModelRetryConfig,
    SummarizationConfig,
    ToolRetryConfig,
)

# Configure agent middleware with custom settings
middleware = AgentMiddlewareConfig(
    limits=AgentLimitConfig(
        model_call_limit=30,  # Max LLM calls per question
        tool_call_limit=60,  # Max tool calls per question
        exit_behavior="end",  # Return partial response on limit
    ),
    model_retry=ModelRetryConfig(
        max_retries=3,  # Retry failed LLM calls
        backoff_factor=2.0,
        initial_delay=2.0,
    ),
    tool_retry=ToolRetryConfig(
        max_retries=3,  # Retry failed tool calls
        on_failure="return_message",  # Return error as message to model
    ),
    summarization=SummarizationConfig(
        enabled=True,  # Summarize when context gets long
        trigger_fraction=0.8,  # Trigger at 80% of context window
        keep_messages=20,  # Keep 20 recent messages after summarization
    ),
)

print(f"Model call limit: {middleware.limits.model_call_limit}")
print(f"Tool call limit: {middleware.limits.tool_call_limit}")
print(f"Model retries: {middleware.model_retry.max_retries}")
print(f"Tool retries: {middleware.tool_retry.max_retries}")
print(f"Summarization: {middleware.summarization.enabled}")

Model call limit: 30
Tool call limit: 60
Model retries: 3
Tool retries: 3
Summarization: True


## Step 3: Run MCP-Enabled Verification

Combine the MCP-configured model with middleware and run verification.
The answering model will use MCP tools to gather information before
responding.

In [ ]:
from karenina.benchmark import Benchmark

benchmark = Benchmark.load("test_checkpoint.jsonld")

# Attach middleware to the answering model
agent_model = ModelConfig(
    id="agent-claude",
    model_name="claude-sonnet-4-5-20250929",
    model_provider="anthropic",
    interface="langchain",
    mcp_urls_dict={"search": "http://localhost:3000/mcp"},
    mcp_tool_filter=["web_search"],
    agent_middleware=middleware,
)

# Parsing model does not need MCP
parsing_model = ModelConfig(
    id="parser-gpt4o",
    model_name="gpt-4o",
    model_provider="openai",
    interface="langchain",
)

config = VerificationConfig(
    answering_models=[agent_model],
    parsing_models=[parsing_model],
)

results = benchmark.run_verification(config)

summary = results.get_summary()
pass_info = summary.get("template_pass_overall", {})
print(f"Total results: {len(results.results)}")
print(f"Passed: {pass_info.get('passed', 0)}/{pass_info.get('total', 0)}")

## Step 4: Inspect Agent Traces

MCP-enabled models produce **agent traces** — sequences of messages including
LLM responses, tool calls, and tool results. The full trace is always captured
in `raw_llm_response`.

In [5]:
for result in results:
    q_text = result.metadata.question_text
    if result.template:
        trace = result.template.raw_llm_response
        passed = result.template.verify_result
        status = "PASS" if passed else "FAIL"
        print(f"\n--- {q_text[:50]}... [{status}] ---")
        # Show first 3 lines of the agent trace
        for line in trace.split("\n")[:3]:
            print(f"  {line}")
    else:
        print(f"\n--- {q_text[:50]}... [NO TEMPLATE] ---")


--- What is the capital of France?... [PASS] ---
  [Agent] Received question: What is the capital of France?
  [Agent] Using tools from: search
  [Tool Call] web_search('relevant query')

--- What is 6 multiplied by 7?... [PASS] ---
  [Agent] Received question: What is 6 multiplied by 7?
  [Agent] Using tools from: search
  [Tool Call] web_search('relevant query')

--- What element has the atomic number 8? Provide both... [PASS] ---
  [Agent] Received question: What element has the atomic number 8? Provide both the name and chemical symbol.
  [Agent] Using tools from: search
  [Tool Call] web_search('relevant query')

--- Is 17 a prime number?... [FAIL] ---
  [Agent] Received question: Is 17 a prime number?
  [Agent] Using tools from: search
  [Tool Call] web_search('relevant query')

--- Explain the concept of machine learning in simple ... [NO TEMPLATE] ---


## Step 5: Handle Recursion Limit Hits

When an agent exceeds `model_call_limit` or `tool_call_limit`, the pipeline
auto-fails the question. Check `recursion_limit_reached` on the template
result to identify these cases.

In [6]:
for result in results:
    if result.template and result.template.recursion_limit_reached:
        print(f"Recursion limit hit: {result.metadata.question_text[:60]}")
        print(f"  verify_result: {result.template.verify_result}")
        print(f"  completed_without_errors: {result.metadata.completed_without_errors}")
        # The trace is still available for analysis
        print(f"  trace lines: {len(result.template.raw_llm_response.split(chr(10)))}")

Recursion limit hit: Is 17 a prime number?
  verify_result: False
  completed_without_errors: True
  trace lines: 6


## Step 6: Trace Input for Evaluation

Two flags on `VerificationConfig` control what evaluation models see:

| Flag | Default | What the Evaluator Sees |
|------|---------|------------------------|
| `use_full_trace_for_template` | `False` | Only the final answer (less noise, lower cost) |
| `use_full_trace_for_rubric` | `True` | Full agent trace (tool usage, reasoning process) |

The full trace is **always** captured regardless of these flags.

In [7]:
# Configure trace input separately for template and rubric evaluation
config_trace = VerificationConfig(
    answering_models=[agent_model],
    parsing_models=[parsing_model],
    # Template parsing: only final answer (default)
    use_full_trace_for_template=False,
    # Rubric evaluation: full agent trace (default)
    use_full_trace_for_rubric=True,
)

print(f"Template sees full trace: {config_trace.use_full_trace_for_template}")
print(f"Rubric sees full trace: {config_trace.use_full_trace_for_rubric}")

# Run with trace settings
trace_results = benchmark.run_verification(config_trace)
print(f"\nResults: {len(trace_results.results)}")

Template sees full trace: False
Rubric sees full trace: True

Results: 5


## Summary

| Feature | Configuration |
|---------|---------------|
| Enable MCP | Set `mcp_urls_dict` on `ModelConfig` |
| Filter tools | Set `mcp_tool_filter` on `ModelConfig` |
| Execution limits | `AgentLimitConfig` in `AgentMiddlewareConfig` |
| Retry behavior | `ModelRetryConfig` and `ToolRetryConfig` |
| Summarization | `SummarizationConfig` (LangChain adapter only) |
| Trace input | `use_full_trace_for_template/rubric` on `VerificationConfig` |

## Next Steps

- [MCP-Enabled Verification](../06-running-verification/mcp-verification.md) — full documentation
- [Adapters Overview](../04-core-concepts/adapters.md) — adapter-specific MCP behavior
- [Python API Verification](../06-running-verification/python-api.md) — standard verification workflow
- [Multi-Model Evaluation](../06-running-verification/multi-model.md) — comparing models

In [8]:
# Cleanup mocks
_ = _patcher1.stop()
_ = _patcher2.stop()